In [117]:
from tqdm.auto import tqdm
import time
import subprocess
import random

In [118]:
wa = False
from typing import List

class Test():
    def __init__(self):
        self.generate_test()
        
    def generate_test(self): 
        with open("in/test.in", 'w') as test:
            self.q = random.randrange(4, 6)
            self.len_now = 0
            
            test.write(f"{self.q}\n")
            for _ in range(self.q):
                if self.len_now == 0: 
                    c = 'I'
                else:
                    c = random.choice(['I', 'D', 'S'])
                    
                if c == 'I':
                    pos = random.randrange(0, self.len_now + 1)
                    x = random.randrange(-1_0, 1_0)
                    
                    self.len_now += 1
                    test.write(f"{c} {pos} {x}\n")
                elif c == 'D':
                    pos = random.randrange(1, self.len_now + 1)
                    
                    self.len_now -= 1
                    test.write(f"{c} {pos}\n")
                else:
                    p1 = random.randrange(1, self.len_now + 1)
                    p2 = random.randrange(p1, self.len_now + 1)
                    test.write(f"{c} {p1} {p2}\n")
        
    def check_solutions(self, a_out: str, b_out: str):
        global wa
        # a: brut, b: better_solution
        with open(a_out, 'r') as a, open(b_out, 'r') as b:
            self.brut_data = [int(i) for i in a.read().strip().split()]
            self.chad_data = [int(i) for i in b.read().strip().split()]
                
        self.better_check()
    
    def better_check(self):
        global wa
        
        if self.brut_data != self.chad_data:
            print("WA")
            wa = True
            return

In [119]:
# convention a: brut, b: better_solutoin
class Referee():
    def __init__(self, program_a: str, program_b: str, max_time: int):
        # program a = sth.cpp (give full name with .cpp)
        
        self.program_a = program_a
        self.program_b = program_b
        self.max_time = max_time
        self.times: List[List[float]] = []
        
        # compile programs 
        self.exe_a = self.compile(program_a)
        self.exe_b = self.compile(program_b)
        
    def compile(self, program: str) -> str:
        exe_name = program.split('.')[0]
        
        compile_command = [
            'g++',
            f'src/{program}',
            '-o', f'bin/{exe_name}',
            '-std=gnu++17',
            '-O2',
        ]
        subprocess.run(compile_command, check=True)
        return exe_name
    
    def do_tests(self): 
        self.times = []
        for _ in tqdm(range(5000)): 
            # create test 
            self.test = Test()
            
            # test program_a
            start_time_a = time.time()
            with open("in/test.in", "r") as test_input, open(f"out/{self.exe_a}.out", "w") as output_a:
                try:
                    subprocess.run(f"./bin/{self.exe_a}", stdin=test_input, stdout=output_a, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_a} exceeded time limit")
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_a} ended with an error")
                    exit(2)
            end_time_a = time.time()
            
            # test program_b
            start_time_b = time.time()
            with open("in/test.in", "r") as test_input, open(f"out/{self.exe_b}.out", "w") as output_b:
                try:
                    subprocess.run(f"./bin/{self.exe_b}", stdin=test_input, stdout=output_b, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_b} exceeded time limit") 
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_b} ended with an error")
                    exit(2)
            end_time_b = time.time()
            
            
            # check solution
            global wa
            wa = False
            self.test.check_solutions(f"out/{self.exe_a}.out", f"out/{self.exe_b}.out")
            if wa:
               break 
            
            # solutions are ok
            self.times.append([end_time_a - start_time_a, end_time_b - start_time_b])
        
        else:
            print("everything ok 🚀") 

In [120]:
referee = Referee('brut_solution.cpp', 'gigachad_solution.cpp', 2)
referee.do_tests()

 53%|█████▎    | 2656/5000 [00:13<00:11, 199.83it/s]

WA


In [121]:
import pandas as pd
df = pd.DataFrame(referee.times)
df.columns = ['brut', 'chad']
df = df.sort_values(by='chad', ascending=False)
df

,brut,chad
311,0.001718,0.005775
1439,0.001457,0.004019
153,0.001404,0.003899
2519,0.003813,0.003792
1805,0.001358,0.003790
...,...,...
2235,0.001465,0.003354
464,0.001326,0.003352
946,0.001373,0.003347
942,0.001312,0.003343


In [122]:
df.mean()
df.sort_values(by='brut', ascending=False)

,brut,chad
2519,0.003813,0.003792
1440,0.001814,0.003598
1443,0.001795,0.003761
311,0.001718,0.005775
1449,0.001688,0.003711
...,...,...
2045,0.001295,0.003403
1275,0.001292,0.003390
1035,0.001288,0.003657
1195,0.001276,0.003404
